# Modelling and Evaluation

## Objectives

- Complete model to predict if a cherry leaf is infected with powdery mildew.

## Inputs

- inputs/cherry_leaves_raw_dataset/cherry-leaves/train
- inputs/cherry_leaves_raw_dataset/cherry-leaves/validation
- inputs/cherry_leaves_raw_dataset/cherry-leaves/test


## Outputs

- Image distribution plot, train vs validation vs test.
- Machine learning model creation and training.
- Plotting model performance.
- Prediction of a selected image file.

## Install dependencies

In [ ]:
pip install -r ./requirements.txt


## Change working directory

Change working directory from current to parent folder.

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection/jupyter_notebooks'

In [2]:
os.chdir("/workspace/mildew-detection")
print("You set a new current directory.")

You set a new current directory.


In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/mildew-detection'

## Set input directories

In [4]:
data_dir = "inputs/cherry_leaves_raw_dataset/cherry-leaves"
train_dir = data_dir + "/train"
val_dir = data_dir + "/validation"
test_dir = data_dir + "/test"

## Set output directories

In [5]:
version = "v1"
file_path = f"outputs/{version}"
if "outputs" in os.listdir(current_dir) and version in os.listdir(current_dir + "/outputs"):
    print("This directory already exists, please create a new version.")
else:
    os.makedirs(name=file_path)

This directory already exists, please create a new version.


## Set label names

In [ ]:
labels = os.listdir(train_dir)
print("The image labels are ", labels)

# Machine Learning Model Creation

Import model packages

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPooling3D